**Step 0: Import Libraries**

In [18]:
from functools import lru_cache
from typing import Dict, Set, List, Iterator, Optional, Tuple
import pandas as pd

**Step 1: Import Showcase Line-UP as CSV**

In [19]:
pathfile = "/Users/iz132/Desktop/f25/mvm_script/F25_LineUp - Sheet1.csv"
df = pd.read_csv(pathfile)

**Step 2: Clean Data from CSV as Needed**

In [20]:
# Function to normalize names
def normalize_name(name):
    """
    Normalize a name to lowercase with underscores.
    Handles common variations and inconsistencies.
    """
    if pd.isna(name) or name == '':
        return name
    
    # Strip whitespace first
    name = str(name).strip()
    
    # Convert to lowercase
    name = name.lower()
    
    # Remove periods
    name = name.replace('.', '')
    
    # Replace spaces with underscores
    name = name.replace(' ', '_')
    
    return name

In [27]:
# Convert all cell values to lowercase
df = df.apply(lambda col: col.str.lower() if col.dtype == "object" else col)

# Strip whitespace from all cell values
df = df.apply(lambda col: col.str.strip() if col.dtype == "object" else col)

# Apply the normalize_name function to all cell values
df = df.apply(lambda col: col.apply(normalize_name) if col.dtype == "object" else col)

**Step 3: Create mappings from CSV to Dictionary (Preprocess)**

In [28]:
# Auto-generate teams_to_members dictionary from cleaned DataFrame

teams_to_members = {}

for col in df.columns:
    members = df[col].dropna()
    members = members[members != '']
    teams_to_members[col] = set(members.unique())

# Print in the same format as your sample_teams_data
print("teams_to_members = {")
for team, members in teams_to_members.items():
    members_str = ', '.join(f'"{m}"' for m in sorted(members))
    print(f'    "{team}": {{{members_str}}},')
print("}")

teams_to_members = {
    "go": {"angie", "justin", "leo_z", "luke", "meso", "sophia_z", "timothy"},
    "nxde": {"angela", "aslan", "eni", "meso", "sherla"},
    "loco": {"adell", "aslan", "ava", "elena", "sua"},
    "guilty": {"joey", "sophia_d", "sophia_z", "vivian", "wei_lun"},
    "bad_villain": {"andy", "eni", "haeun", "hairuo", "mandy", "meso", "vivian"},
    "last_festival": {"andy", "brandon", "irene", "max", "roxanne", "sean"},
    "hot": {"ava", "neha", "roxanne", "sarea", "sua"},
    "famous": {"elena", "hairuo", "joey", "kaiki", "leo_shen"},
    "dirty_work": {"aslan", "hairuo", "kaiki", "sabrina"},
    "drip": {"irene", "irving", "phuong", "talia", "yuna"},
    "plot_twist": {"andrew_liu", "brandon", "leo_shen", "lisa", "sophia_z", "timothy"},
    "xoxz": {"andrew_lee", "aslan", "chunzhen", "irene", "sarea", "talia"},
    "siren": {"adell", "chi", "kaiki", "kaylee", "phuong", "sean", "talia"},
    "grabriela": {"angela", "haeun", "michael", "sherla", "vivian"},
    "jellyo

**Step 4: Run Script to Find Optimal Order**

In [29]:
class SetlistSolver:
    """
    Finds and counts valid sequences of teams (setlists) where no two
    adjacent teams share a member (Hamiltonian paths in a compatibility graph).
    """

    def __init__(self, teams_to_members: Dict[str, Set[str]]):
        if not teams_to_members:
            raise ValueError("Input `teams_to_members` cannot be empty.")

        # Light sanitization: strip whitespace around member names
        self.teams_to_members: Dict[str, Set[str]] = {
            team: {str(m).strip() for m in members}
            for team, members in teams_to_members.items()
        }

        self.teams: List[str] = list(self.teams_to_members.keys())
        self.team_count: int = len(self.teams)
        self.team_to_idx: Dict[str, int] = {team: i for i, team in enumerate(self.teams)}
        self.adjacency_masks: List[int] = self._build_compatibility_graph()

    def _build_compatibility_graph(self) -> List[int]:
        """Bitmask adjacency: adj[i] has bit j set iff team i is compatible with j."""
        adj = [0] * self.team_count
        for i, team_i in enumerate(self.teams):
            members_i = self.teams_to_members[team_i]
            mask = 0
            for j, team_j in enumerate(self.teams):
                if i == j:
                    continue
                if self.teams_to_members[team_j].isdisjoint(members_i):
                    mask |= (1 << j)
            adj[i] = mask
        return adj

    # ---------- Generate valid setlists (optional start/end constraints) ----------

    def generate_valid_setlists(
        self,
        limit: Optional[int] = None,
        start_team: Optional[str] = None,
        end_team: Optional[str] = None,
        debug: bool = False,
    ) -> Iterator[List[str]]:
        """
        Backtracking + heuristics. Before yielding, we STRICTLY validate that
        each adjacent pair shares no members (guarding against data issues).
        """
        if start_team and start_team not in self.team_to_idx:
            raise ValueError(f"Start team '{start_team}' not found.")
        if end_team and end_team not in self.team_to_idx:
            raise ValueError(f"End team '{end_team}' not found.")

        # Heuristic: try nodes with fewer neighbors first (tighter first)
        ranked_indices = sorted(
            range(self.team_count),
            key=lambda i: bin(self.adjacency_masks[i]).count("1")
        )
        idx_to_rank = {orig: r for r, orig in enumerate(ranked_indices)}

        # Reindex adjacency + names according to ranking
        adj_r = [0] * self.team_count
        teams_r = [self.teams[i] for i in ranked_indices]
        for i in range(self.team_count):
            original_mask = self.adjacency_masks[i]
            m = 0
            for j in range(self.team_count):
                if (original_mask >> j) & 1:
                    m |= (1 << idx_to_rank[j])
            adj_r[idx_to_rank[i]] = m

        start_node_r = idx_to_rank[self.team_to_idx[start_team]] if start_team else None
        end_node_r   = idx_to_rank[self.team_to_idx[end_team]]   if end_team   else None

        produced = 0
        for path_indices in self._backtrack_with_end(adj_r, start_node_r, end_node_r):
            if limit is not None and produced >= limit:
                return

            # Convert to team names
            seq = [teams_r[i] for i in path_indices]

            # STRICT VALIDATION: verify every adjacent pair has disjoint members
            ok, conflict = self._sequence_is_valid(seq)
            if not ok:
                if debug:
                    a, b, overlap = conflict
                    print(f"[skip invalid] {a} -> {b} shares members: {sorted(overlap)}")
                continue

            yield seq
            produced += 1

    def _backtrack_with_end(
        self,
        adj_r: List[int],
        start_node_r: Optional[int],
        end_node_r: Optional[int],
    ) -> Iterator[List[int]]:
        """
        Enforce fixed end by forbidding `end_node_r` until last slot and
        forcing it at the final step (if reachable).
        """
        n = self.team_count
        path: List[int] = []
        used_mask = 0

        def solve():
            nonlocal used_mask
            L = len(path)
            if L == n:
                if end_node_r is None or path[-1] == end_node_r:
                    yield path.copy()
                return

            if L == 0:
                candidates = [start_node_r] if start_node_r is not None else list(range(n))
            else:
                last = path[-1]
                allowed = adj_r[last] & ~used_mask

                if end_node_r is not None and L < n - 1:
                    allowed &= ~(1 << end_node_r)

                if end_node_r is not None and L == n - 1:
                    if (allowed >> end_node_r) & 1:
                        allowed = (1 << end_node_r)
                    else:
                        return  # cannot reach required end

                candidates = []
                x = allowed
                while x:
                    lsb = x & -x
                    candidates.append(lsb.bit_length() - 1)
                    x ^= lsb

                # heuristic: fewest future options first
                candidates.sort(key=lambda c: bin(adj_r[c] & ~used_mask).count("1"))

            for c in candidates:
                if (used_mask >> c) & 1:
                    continue
                path.append(c)
                used_mask |= (1 << c)
                yield from solve()
                used_mask &= ~(1 << c)
                path.pop()

        yield from solve()

    # ---------- Validation & Debug helpers ----------

    def _sequence_is_valid(self, seq: List[str]) -> Tuple[bool, Optional[Tuple[str, str, Set[str]]]]:
        """Return (True, None) if valid; else (False, (teamA, teamB, overlapping_members))."""
        for a, b in zip(seq, seq[1:]):
            overlap = self.teams_to_members[a] & self.teams_to_members[b]
            if overlap:
                return False, (a, b, overlap)
        return True, None

    def explain_pair(self, team_a: str, team_b: str) -> None:
        """Print the intersection (if any) for a quick manual check."""
        inter = self.teams_to_members[team_a] & self.teams_to_members[team_b]
        if inter:
            print(f"{team_a} and {team_b} share: {sorted(inter)}")
        else:
            print(f"{team_a} and {team_b} are compatible (no shared members).")


In [30]:
# Initialize the solver
solver = SetlistSolver(teams_to_members)

# Quick sanity check (uncomment to use):
# solver.explain_pair("plot_twist", "famous")

print(f"Working with {solver.team_count} teams.")


Working with 16 teams.


## Step 5: Find Setlists with Constraints

Find setlists with fixed start, end, and multiple fixed middle team positions.


In [31]:
# Define all of your constraints here
start = "go"
end = "dope"

# First positional constraint
target_team_1 = "xoxz"
target_position_index_1 = 8

# Second positional constraint
target_team_2 = "bad_villain"
target_position_index_2 = 7

# Third positional constraint
target_team_3 = "siren"
target_position_index_3 = 11

# Fourth positional constraint
target_team_4 = "drip"
target_position_index_4 = 14


In [32]:
# Run the search
try:
    print(
        f"\n--- Finding setlists starting with '{start}', ending with '{end}', "
        f"with '{target_team_1}' in position {target_position_index_1 + 1}, "
        f"'{target_team_2}' in position {target_position_index_2 + 1}, "
        f"'{target_team_3}' in position {target_position_index_3 + 1}, "
        f"and '{target_team_4}' in position {target_position_index_4 + 1} ---"
    )

    found_count = 0
    limit = 10  # Find up to 10 examples

    # Call the generator with the start and end teams locked
    generator = solver.generate_valid_setlists(
        start_team=start,
        end_team=end
    )

    # Loop through the results and apply all positional checks
    for setlist in generator:
        # Check all four positional constraints
        if (
            len(setlist) > target_position_index_1
            and setlist[target_position_index_1] == target_team_1
            and len(setlist) > target_position_index_2
            and setlist[target_position_index_2] == target_team_2
            and len(setlist) > target_position_index_3
            and setlist[target_position_index_3] == target_team_3
            and len(setlist) > target_position_index_4
            and setlist[target_position_index_4] == target_team_4
        ):
            found_count += 1
            print(f"{found_count}: {setlist}")

        if found_count >= limit:
            break

    if found_count == 0:
        print("No valid setlists found that satisfy all specified conditions.")

except ValueError as e:
    print(f"Error: {e}")



--- Finding setlists starting with 'go', ending with 'dope', with 'xoxz' in position 9, 'bad_villain' in position 8, 'siren' in position 12, and 'drip' in position 15 ---
1: ['go', 'last_festival', 'famous', 'nxde', 'jellyous', 'loco', 'plot_twist', 'bad_villain', 'xoxz', 'guilty', 'hot', 'siren', 'grabriela', 'dirty_work', 'drip', 'dope']
2: ['go', 'last_festival', 'famous', 'nxde', 'jellyous', 'loco', 'plot_twist', 'bad_villain', 'xoxz', 'grabriela', 'hot', 'siren', 'guilty', 'dirty_work', 'drip', 'dope']
3: ['go', 'last_festival', 'famous', 'nxde', 'jellyous', 'guilty', 'loco', 'bad_villain', 'xoxz', 'plot_twist', 'grabriela', 'siren', 'hot', 'dirty_work', 'drip', 'dope']
4: ['go', 'last_festival', 'famous', 'nxde', 'jellyous', 'guilty', 'loco', 'bad_villain', 'xoxz', 'plot_twist', 'hot', 'siren', 'grabriela', 'dirty_work', 'drip', 'dope']
5: ['go', 'last_festival', 'famous', 'nxde', 'jellyous', 'guilty', 'loco', 'bad_villain', 'xoxz', 'grabriela', 'plot_twist', 'siren', 'hot', 'di